# 11 - AgriPV Systems

This journal shows how to model an AgriPV site, calculating the irradiance not only on the modules but also the irradiance received by the ground to evaluate available solar ersource for plants. 

We assume that bifacia_radiacne is already installed in your computer. This works for bifacial_radiance v.3 release.

These journal outlines 4 useful uses of bifacial_radiance and some tricks: 

* Creating the modules in the AgriPV site
* Adding extra geometry for the pillars/posts supporting the AgriPV site
* Hacking the sensors to sample the ground irradiance and create irradiance map
* Adding object to simulate variations in ground albedo from different crops between rows.


#### Steps:

1. <a href='#step1'> Generate the geometry </a>
2. <a href='#step2'> Analyse the Ground Irradiance </a>
3. <a href='#step3'> Analyse and MAP the Ground Irradiance </a>
4. <a href='#step4'> Adding different Albedo Section </a>
    
#### Preview of what we will create: 
    
![Another view](../images_wiki/AdvancedJournals/AgriPV_2.PNG)
![AgriPV Image We will create](../images_wiki/AdvancedJournals/AgriPV_1.PNG)
And this is how it will look like:

![AgriPV modeled step 4](../images_wiki/AdvancedJournals/AgriPV_step4.PNG)




<a id='step1'></a>

## 1. Generate the geometry 

This section goes from setting up variables to making the OCT axis. We are also adding some custom elements for the torquetubes and posts.

We've done this before a couple times, no new stuff here. 

The magic is that, for doing the carport we see in the figure, we are going to do a 4-up configuration of modules (**numpanels**), and we are going to repeat that 3-UP 6 times (**nMods**)

In [1]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent / 'TEMP' /  'Tutorial_1')

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\InSPIRE\TEMP\Tutorial_1


In [2]:
import bifacial_radiance as br
import numpy as np
import pandas as pd

In [3]:
simulationname = 'tutorial_1'

#Location:
lat = 40.0583  # NJ
lon = -74.4057  # NJ

# MakeModule Parameters
moduletype='test-module'
numpanels = 3  # AgriPV site has 3 modules along the y direction (N-S since we are facing it to the south) .
x = 1  
y = 2
xgap = 2.0# Leaving 15 centimeters between modules on x direction
ygap = 0.10 # Leaving 10 centimeters between modules on y direction
zgap = 0 # no gap to torquetube.
sensorsy = 6*numpanels  # this will give 6 sensors per module, 1 per cell

# Other default values:

# TorqueTube Parameters
axisofrotationTorqueTube=False  # this is False by default if there is no torquetbue parameters
torqueTube = False
cellLevelModule = True

numcellsx = 12
numcellsy = 6
xcell = 0.156
ycell = 0.156
xcellgap = 0.02
ycellgap = 0.02

cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

# SceneDict Parameters
pitch = 15 # m
albedo = 0.2  #'grass'     # ground albedo
hub_height = 4.3 # m  
nMods = 6 # six modules per row.
nRows = 3  # 3 row

azimuth_ang=180 # Facing south
tilt =35 # tilt.

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)  
demo.setGround(albedo) 
epwfile = demo.getEPW(lat, lon) # NJ lat/lon 40.0583° N, 74.4057
# Making module with all the variables
module=demo.makeModule(name=moduletype,x=x,y=y,numpanels=numpanels, 
                           xgap=xgap, ygap=ygap, cellModule=cellLevelModuleParams)
# create a scene with all the variables


path = C:\Users\sayala\Documents\GitHub\InSPIRE\TEMP\Tutorial_1
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_NJ_McGuire.AFB.724096_TMY3.epw
 ... OK!


Exception: incorrect time string passed 2021-06-17 13:0:0 -5.Valid options: mm_dd, mm_dd_HH, mm_dd_HHMM, YYYY-mm-dd_HHMM

In [6]:
#startime and endtime 
#Valid options: mm_dd, mm_dd_HH, mm_dd_HHMM, YYYY-mm-dd_HHMM
metdata = demo.readWeatherFile(epwfile, coerce_year=2021, starttime='2021-06-17_1300', endtime='2021-06-17_1500')


8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [10]:
metdata.__dict__

{'latitude': 40.02,
 'longitude': -74.6,
 'elevation': 45.0,
 'timezone': -5.0,
 'city': 'Mcguire Afb',
 'datetime': [Timestamp('2021-06-17 13:00:00-0500', tz='UTC-05:00'),
  Timestamp('2021-06-17 14:00:00-0500', tz='UTC-05:00'),
  Timestamp('2021-06-17 15:00:00-0500', tz='UTC-05:00')],
 'ghi': array([764, 629, 528], dtype=int64),
 'dhi': array([437, 478, 325], dtype=int64),
 'dni': array([344, 166, 250], dtype=int64),
 'albedo': array([0.14, 0.14, 0.14]),
 'dewpoint': array([18.3, 18.9, 18.3]),
 'pressure': array([100700, 100700, 100700], dtype=int64),
 'temp_air': array([32.2, 31.7, 32.8]),
 'wind_speed': array([1.5, 1. , 1. ]),
 'meastracker_angle': None,
 'solpos':                            apparent_zenith     zenith  apparent_elevation  \
 corrected_timestamp                                                         
 2021-06-17 12:30:00-05:00        17.825584  17.830959           72.174416   
 2021-06-17 13:30:00-05:00        25.296241  25.304152           64.703759   
 2021-06-17

In [17]:
timeindex = metdata.datetime.index(pd.to_datetime('2021-06-17 13:0:0 -5'))  # Make this timezone aware, use -5 for EST.
timeindex

0

In [18]:
demo.gendaylit(timeindex=timeindex)  

'skies\\sky2_40.02_-74.6_2021-06-17_1300.rad'

In [20]:
sceneDict = {'tilt':tilt,'pitch': 15,'hub_height':hub_height,'azimuth':azimuth_ang, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(module=moduletype, sceneDict=sceneDict) 
octfile = demo.makeOct(demo.getfilelist())


Created tutorial_1.oct


If desired, you can view the Oct file at this point:

***rvu -vf views\front.vp -e .01 tutorial_1.oct***

In [21]:
## Comment the ! line below to run rvu from the Jupyter notebook instead of your terminal.
## Simulation will stop until you close the rvu window

!rvu -vf views\front.vp -e .01 tutorial_1.oct


And adjust the view parameters, you should see this image.

![AgriPV modeled step 1](../images_wiki/AdvancedJournals/AgriPV_step1.PNG)


### Adding a torquetube structure

We will add on the torquetube and pillars.

Positions of the piles could be done more programatically, but they are kinda estimated at the moment. 

In [ ]:
module=demo.makeModule(name=moduletype,x=x,y=y,numpanels=numpanels, 
                           xgap=xgap, ygap=ygap, cellModule=cellLevelModuleParams, 
                      torquetube=True, )
# create a scene with all the variables

## 2. Analyse  the Ground Irradiance

Now let's do some analysis along the ground, starting from the edge of the modules. We wil select to start in the center of the array.

We are also increasign the number of points sampled accross the collector width, with the  variable **sensorsy** passed to **moduleanalysis**. We are also increasing the step between sampling points, to be able to sample in between the rows.

In [25]:
analysis = br.AnalysisObj(octfile, demo.name)  
sensorsy = 20
frontscan, backscan = analysis.moduleAnalysis(scene, sensorsy=sensorsy)


In [26]:
groundscan = frontscan

In [27]:
groundscan['zstart'] = 0.05  # setting it 5 cm from the ground.
groundscan['zinc'] = 0   # no tilt necessary. 
groundscan['yinc'] = pitch/(sensorsy-1)   # increasing spacing so it covers all distance between rows
groundscan

{'xstart': 1.5159739062663865e-16,
 'ystart': -1.237886635821746,
 'zstart': 0.05,
 'xinc': -1.5802346387185998e-17,
 'yinc': 0.7894736842105263,
 'zinc': 0,
 'sx_xinc': 0.0,
 'sx_yinc': 0.0,
 'sx_zinc': 0.0,
 'Nx': 1,
 'Ny': 20,
 'Nz': 1,
 'orient': '-0.000 0.574 -0.819'}

In [28]:
analysis.analysis(octfile, simulationname+"_groundscan", groundscan, backscan)  # compare the back vs front irradiance  


Linescan in process: tutorial_1_groundscan_Row2_Module3_Front
Linescan in process: tutorial_1_groundscan_Row2_Module3_Back
Saved: results\irr_tutorial_1_groundscan_Row2_Module3.csv


({'Wm2': [705.9618,
   674.8684,
   331.1048,
   331.4625,
   347.4178,
   360.4403,
   709.1403,
   720.8762,
   731.803,
   737.6065,
   741.9417,
   746.2793,
   749.7294,
   750.6525,
   752.3912,
   753.1805,
   747.7246,
   740.8768999999999,
   728.2795,
   704.9231],
  'x': [1.515974e-16,
   1.35795e-16,
   1.199927e-16,
   1.041904e-16,
   8.838801e-17,
   7.258566e-17,
   5.678331e-17,
   4.098097e-17,
   2.517862e-17,
   9.376273e-18,
   -6.426073e-18,
   -2.222842e-17,
   -3.803077e-17,
   -5.383311e-17,
   -6.963546e-17,
   -8.543781e-17,
   -1.012402e-16,
   -1.170425e-16,
   -1.328448e-16,
   -1.486472e-16],
  'y': [-1.237887,
   -0.448413,
   0.3410607,
   1.130534,
   1.920008,
   2.709482,
   3.498955,
   4.288429,
   5.077903,
   5.867377,
   6.65685,
   7.446324,
   8.235798,
   9.025271,
   9.814745,
   10.60422,
   11.39369,
   12.18317,
   12.97264,
   13.76211],
  'z': [0.05,
   0.05,
   0.05,
   0.05,
   0.05,
   0.05,
   0.05,
   0.05,
   0.05,
   0.05,
   0.0

This is the result for only one 'chord' accross the ground. Let's now do a X-Y scan of the ground.

<a id='step3'></a>

## 3. Analyse and MAP the Ground Irradiance

 We will use the same technique to find the irradiance on the ground used above, but will move it along the X-axis to map from the start of one module to the next.
 
 We will sample around the module that is placed at the center of the field.

![AgriPV modeled step 4](../images_wiki/AdvancedJournals/spacing_between_modules.PNG)

In [29]:
import seaborn as sns

In [30]:
sensorsx = 20
startgroundsample=-module.scenex
spacingbetweensamples = module.scenex/(sensorsx-1)

for i in range (0, sensorsx): # Will map 20 points    
    frontscan, backscan = analysis.moduleAnalysis(scene, sensorsy=sensorsy)
    groundscan = frontscan
    groundscan['zstart'] = 0.05  # setting it 5 cm from the ground.
    groundscan['zinc'] = 0   # no tilt necessary. 
    groundscan['yinc'] = pitch/(sensorsy-1)   # increasing spacing so it covers all distance between rows
    groundscan['xstart'] = startgroundsample + i*spacingbetweensamples   # increasing spacing so it covers all distance between rows
    analysis.analysis(octfile, simulationname+"_groundscan_"+str(i), groundscan, backscan)  # compare the back vs front irradiance  


Linescan in process: tutorial_1_groundscan_0_Row2_Module3_Front
Linescan in process: tutorial_1_groundscan_0_Row2_Module3_Back
Saved: results\irr_tutorial_1_groundscan_0_Row2_Module3.csv
Linescan in process: tutorial_1_groundscan_1_Row2_Module3_Front
Linescan in process: tutorial_1_groundscan_1_Row2_Module3_Back
Saved: results\irr_tutorial_1_groundscan_1_Row2_Module3.csv
Linescan in process: tutorial_1_groundscan_2_Row2_Module3_Front
Linescan in process: tutorial_1_groundscan_2_Row2_Module3_Back
Saved: results\irr_tutorial_1_groundscan_2_Row2_Module3.csv
Linescan in process: tutorial_1_groundscan_3_Row2_Module3_Front
Linescan in process: tutorial_1_groundscan_3_Row2_Module3_Back
Saved: results\irr_tutorial_1_groundscan_3_Row2_Module3.csv
Linescan in process: tutorial_1_groundscan_4_Row2_Module3_Front
Linescan in process: tutorial_1_groundscan_4_Row2_Module3_Back
Saved: results\irr_tutorial_1_groundscan_4_Row2_Module3.csv
Linescan in process: tutorial_1_groundscan_5_Row2_Module3_Front
L

Read all the files generated into one dataframe

In [31]:
filestarter = "irr_AgriPV_groundscan_"

filelist = sorted(os.listdir(os.path.join(testfolder, 'results')))
prefixed = [filename for filename in filelist if filename.startswith(filestarter)]
arrayWm2Front = []
arrayWm2Back = []
arrayMatFront = []
arrayMatBack = []
filenamed = []
faillist = []

print('{} files in the directory'.format(filelist.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.

for i in range (0, len(prefixed)):
    ind = prefixed[i].split('_')

    try:
        resultsDF = load.read1Result(os.path.join(testfolder, 'results', prefixed[i]))
        arrayWm2Front.append(list(resultsDF['Wm2Front']))
        arrayWm2Back.append(list(resultsDF['Wm2Back']))
        arrayMatFront.append(list(resultsDF['mattype']))
        arrayMatBack.append(list(resultsDF['rearMat']))
        filenamed.append(prefixed[i])
    except:
        print(" FAILED ", i, prefixed[i])
        faillist.append(prefixed[i])

resultsdf = pd.DataFrame(list(zip(arrayWm2Front, arrayWm2Back, 
                                  arrayMatFront, arrayMatBack)),
                         columns = ['br_Wm2Front', 'br_Wm2Back', 
                                    'br_MatFront', 'br_MatBack'])
resultsdf['filename'] = filenamed

21 files in the directory
0 groundscan files in the directory


Creating a new dataframe where  each element in the front irradiance list is a column. Also transpose and reverse so it looks like a top-down view of the ground.

In [ ]:
df3 = pd.DataFrame(resultsdf['br_Wm2Front'].to_list())
reversed_df = df3.T.iloc[::-1]

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
# Plot
ax = sns.heatmap(reversed_df)
ax.set_yticks([])
ax.set_xticks([])
ax.set_ylabel('')  
ax.set_xlabel('')  
print('')

<a id='step4'></a>